In [1]:
# Autoreload extension
%load_ext autoreload
%autoreload 2

In [2]:
# Add path to geostacks
import sys, os
#sys.path.insert(0, '../../')

In [3]:
#sys.path.insert(0, os.path.abspath('../../_build/pip_packages/lib/python3.8/site-packages/'))

In [4]:
# Load geostacks
from geostacks import SpatialIndexLS8

# Load UI components
import ipyleaflet as ilfl
import ipywidgets as iwg

In [5]:
ls8_index = SpatialIndexLS8('../../LANDSAT_8_C1_cornerpoints.csv')
ls8_index.read()
print(ls8_index.footprint)
print('------------------')

query_pt = [-50., 69.]
idxs = ls8_index.query_pathrow(69, -50)
print(ls8_index.corner_pts_df.iloc[idxs])

None
------------------
      path  row    lat_CTR    lon_CTR     lat_UL     lon_UL     lat_UR  \
695      8   11  69.606470 -47.813174  70.756261 -49.045943  69.997303   
696      8   12  68.279699 -49.506130  69.419648 -50.743468  68.700888   
782      9   11  69.606482 -49.355231  70.757156 -50.589565  69.997520   
783      9   12  68.279697 -51.046291  69.420484 -52.285488  68.700913   
866     10   11  69.606439 -50.895321  70.756467 -52.132700  69.996319   
6845    81  233  69.606467 -48.202833  69.158069 -44.965508  68.442808   
6915    82  233  69.606440 -49.749759  69.160805 -46.513479  68.444886   
7017    83  232  68.279726 -49.604053  67.806027 -46.581815  67.125546   
7018    83  233  69.606447 -51.293647  69.158697 -48.052981  68.442311   
7087    84  232  68.279691 -51.145138  67.808695 -48.124313  67.127863   

         lon_UR     lat_LL     lon_LL     lat_LR     lon_LR  
695  -44.451408  69.154025 -51.050718  68.439813 -46.725544  
696  -46.367352  67.801954 -52.529493

In [28]:
class geostacks_ui():
    
    def __init__(self, zoom=4, lon=-50.,lat=69., spatial_index=None):
        
        self.zoom = zoom
        self.lat = lat
        self.lon = lon
        self.mainmap = None
        self.marker = None
        self.idxs = None
        self.ui_title = None
        self.prlist = []
        self.menuleft = None     # path / row menu
        self.pr_selection = None
        self.map_polygon = None
        self.spatial_index = spatial_index
        self.output = None       # print message output (unused for now)
        self.results = None      # store results (unused for now)
        
    def init_panelleft(self):
        self.ui_title = iwg.HTML("<h2>Drag the marker to your region of interest</h2>")
        self.idxs = self.spatial_index.query_pathrow(self.lat,self.lon)
        self.prlist = [('{:03d}/{:03d}'.format(self.spatial_index.corner_pts_df.iloc[i, 'path'], 
                                               self.spatial_index.corner_pts_df.iloc[i, 'row']) ,
                        i) for i in self.idxs]
        self.menuleft = iwg.Select(options=self.prlist, description='Path/Row:', rows=15)
        
    def init_map(self):
        self.mainmap = ilfl.Map(basemap=ilfl.basemaps.Gaode.Satellite,
                                center=[self.lat, self.lon], zoom=self.zoom)
        self.marker = ilfl.Marker(location=[self.lat, self.lon], draggable=True)
        self.mainmap.add_layer(self.marker)
        self.pr_selection = self.idxs[0]
        self.map_polygon = self.spatial_index.corner_pts_df.iloc[self.pr_selection]
        self.mainmap.add_layer(self.map_polygon)
        
    def gen_ui(self, spatial_index=None):
        if self.spatial_index is None:
            self.spatial_index = spatial_index
        
        self.init_panelleft()
        self.init_map()
        
        self.marker.observe(self._on_location_changed, 'location')
        self.menuleft.observe(self._on_menuleft_selection_changed, names='value')
        leftside = iwg.VBox([self.ui_title, self.menuleft])
        leftside.layout.align_items = 'center'
        return iwg.AppLayout(left_sidebar=leftside, center=self.mainmap)

    # ==== leftmenu update when map marker loc changes
    
    def _on_location_changed(self, event):
        self.query_pt = [self.marker.location[-1], self.marker.location[0]]
        self.idxs = self.spatial_index.query_pathrow(self.lat, self.lon)
        self.prlist = [('{:03d}/{:03d}'.format(self.spatial_index.corner_pts_df.iloc[i, 'path'], 
                                               self.spatial_index.corner_pts_df.iloc[i, 'row']) ,i) for i in self.idxs]
        self.menuleft.options = self.prlist
        
    # ==== map polygon update when leftmenu selection changes

    def _on_menuleft_selection_changed(self, change):
        self.pr_selection = change['new']
        self.map_polygon=self.spatial_index.corner_pts_df.loc[self.pr_selection]


In [29]:
cpanel = geostacks_ui(spatial_index=ls8_index)
cpanel.gen_ui()

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types